In [1]:
import findspark
findspark.init('/home/osboxes/spark-2.4.3-bin-hadoop2.7')
import pyspark
import os

In [2]:
myPath = os.path.join('/home', 'osboxes', 'CourseMaterial', 'Spark_for_Machine_Learning', 'Clustering')

print(myPath)

os.chdir(myPath)

/home/osboxes/CourseMaterial/Spark_for_Machine_Learning/Clustering


In [3]:
os.listdir()

['Clustering_Consulting_Project.ipynb',
 'Clustering Code Along.ipynb',
 'seeds_dataset.csv',
 'seeds_dataset.txt',
 '.ipynb_checkpoints',
 'Clustering_Code_Example.ipynb',
 'Clustering_Consulting_Project_SOLUTIONS.ipynb',
 'hack_data.csv',
 'sample_kmeans_data.txt']

In [4]:
# set up the session info

from pyspark.sql import SparkSession

name = 'cluster'

spark = SparkSession.builder.appName(name).getOrCreate()

from pyspark.ml.clustering import KMeans

In [5]:
# get the data

dataset = spark.read.csv("seeds_dataset.csv",header=True,inferSchema=True)
dataset.show()

+-----+---------+-----------+------------------+------------------+---------------------+------------------+
| area|perimeter|compactness|  length_of_kernel|   width_of_kernel|asymmetry_coefficient|  length_of_groove|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+
|15.26|    14.84|      0.871|             5.763|             3.312|                2.221|              5.22|
|14.88|    14.57|     0.8811| 5.553999999999999|             3.333|                1.018|             4.956|
|14.29|    14.09|      0.905|             5.291|3.3369999999999997|                2.699|             4.825|
|13.84|    13.94|     0.8955|             5.324|3.3789999999999996|                2.259|             4.805|
|16.14|    14.99|     0.9034|5.6579999999999995|             3.562|                1.355|             5.175|
|14.38|    14.21|     0.8951|             5.386|             3.312|   2.4619999999999997|             4.956|
|14.69|    14.49|  

In [6]:
dataset.describe().show()

+-------+------------------+------------------+--------------------+-------------------+------------------+---------------------+-------------------+
|summary|              area|         perimeter|         compactness|   length_of_kernel|   width_of_kernel|asymmetry_coefficient|   length_of_groove|
+-------+------------------+------------------+--------------------+-------------------+------------------+---------------------+-------------------+
|  count|               210|               210|                 210|                210|               210|                  210|                210|
|   mean|14.847523809523816|14.559285714285718|  0.8709985714285714|  5.628533333333335| 3.258604761904762|   3.7001999999999997|  5.408071428571429|
| stddev|2.9096994306873647|1.3059587265640225|0.023629416583846364|0.44306347772644983|0.3777144449065867|   1.5035589702547392|0.49148049910240543|
|    min|             10.59|             12.41|              0.8081|              4.899|            

In [7]:
dataset.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)



In [8]:
# import the assembler

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [9]:
dataset.columns

['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [10]:
vec_assembler = VectorAssembler(inputCols = dataset.columns, outputCol='features')

In [11]:
# now transform it

final_data = vec_assembler.transform(dataset)

In [12]:
final_data.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)
 |-- features: vector (nullable = true)



In [13]:
# scale the data

from pyspark.ml.feature import StandardScaler

In [14]:
# use the default scaler

scaler = StandardScaler(inputCol='features',
                       outputCol='scaledFeatures',)

In [15]:
# given the data, scaling won't improve things too much

scaler_model = scaler.fit(final_data)
final_data = scaler_model.transform(final_data)

In [16]:
final_data.head(1)

[Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), scaledFeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))]

In [17]:
kmeans = KMeans(featuresCol='scaledFeatures', k=3)

In [19]:
model = kmeans.fit(final_data)

In [21]:
print('WSSE')
print(model.computeCost(final_data))

WSSE
428.60820118716356


In [22]:
centers = model.clusterCenters()
print(centers)

[array([ 4.07497225, 10.14410142, 35.89816849, 11.80812742,  7.54416916,
        3.15410901, 10.38031464]), array([ 6.35645488, 12.40730852, 37.41990178, 13.93860446,  9.7892399 ,
        2.41585013, 12.29286107]), array([ 4.96198582, 10.97871333, 37.30930808, 12.44647267,  8.62880781,
        1.80061978, 10.41913733])]


In [23]:
model.transform(final_data).show()

+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+--------------------+----------+
| area|perimeter|compactness|  length_of_kernel|   width_of_kernel|asymmetry_coefficient|  length_of_groove|            features|      scaledFeatures|prediction|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+--------------------+----------+
|15.26|    14.84|      0.871|             5.763|             3.312|                2.221|              5.22|[15.26,14.84,0.87...|[5.24452795332028...|         2|
|14.88|    14.57|     0.8811| 5.553999999999999|             3.333|                1.018|             4.956|[14.88,14.57,0.88...|[5.11393027165175...|         2|
|14.29|    14.09|      0.905|             5.291|3.3369999999999997|                2.699|             4.825|[14.29,14.09,0.90...|[4.91116018695588...|         2|
|13.84|    13.94|     0.8955

In [24]:
model.transform(final_data).select('prediction').show()

+----------+
|prediction|
+----------+
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         1|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         0|
+----------+
only showing top 20 rows

